In [105]:
import pandas as pd

In [106]:
import numpy as np

In [107]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import joblib #For model serialization
from flask import Flask, request, jsonify

In [108]:
df=pd.read_csv(r"C:\Users\nirma\Downloads\advertising (2).csv")

In [109]:
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [110]:
df.describe()

,TV,radio,newspaper,sales
count,200.000000,200.000000,200.000000,200.000000
mean,147.042500,23.264000,30.554000,14.022500
std,85.854236,14.846809,21.778621,5.217457
min,0.700000,0.000000,0.300000,1.600000
25%,74.375000,9.975000,12.750000,10.375000
50%,149.750000,22.900000,25.750000,12.900000
75%,218.825000,36.525000,45.100000,17.400000
max,296.400000,49.600000,114.000000,27.000000


In [111]:
X=df.drop("sales", axis=1)
Y=df["sales"]
X_train, X_test,Y_train, Y_test=train_test_split(X,Y, test_size=0.2,random_state=42)

In [112]:
# StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [113]:
# 3. Build the deep learning model
model=Sequential()
model.add(Dense(64, activation="relu",input_dim=X_train.shape[1]))#hidden layer 1
model.add(Dense(32, activation="relu"))#hidden layer 2
model.add(Dense(1)) # output layer

C:\Users\nirma\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [114]:
#compile the model
model.compile(optimizer=Adam(), loss="mean_squared_error")

In [115]:
# 4.Train the model
model.fit(X_train, Y_train, epochs=50, batch_size=32,verbose=1)

Epoch 1/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 222.6247
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 213.2848 
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 214.3566 
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 206.4361 
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 204.1171 
Epoch 6/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 187.4328 
Epoch 7/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 197.4018 
Epoch 8/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 174.0781 
Epoch 9/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 178.8395 
Epoch 10/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 158.4946 
Epoch 11/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 156.5467 
Epoch 12/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 145.4770 
Epoch 13/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 134.4260 
Epoch 14/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 116.9444 
Epoch 15/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 110.0599 
Epoch

In [116]:
loss = model.evaluate(X_test,Y_test)
print("Test Loss:", loss)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 11.3864  
Test Loss: 11.510210037231445


In [117]:
joblib.dump(model, "regression_model.pkl")
joblib.dump(scaler,"scaler.pkl")

['scaler.pkl']

In [118]:
app=Flask(__name__)

In [119]:
model = joblib.load("regression_model.pkl")
scaler = joblib.load("scaler.pkl")

In [120]:
 @app.route("/predict", methods=["POST"])
def predict():
    data = request.get_json()
    input_data = pd.DataFrame([data])
    input_data_scaled = scaler.transform(input_data)
    prediction = model.predict(input_data_scaled)[0][0]
    return jsonify({"prediction":prediction})

In [126]:
 if __name__ == "__main__":
     app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\nirma\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
